In [39]:
import numpy as np
import pandas as pd


In [137]:
a = list(range(1,51))
b = ['-', '*', "/", '+']
popSize = 10
muChance = 0.05
parentEligibility = 0.3
target = 555
MaxGeneration = 250

In [138]:
def generatePoulation(a,b,n):
    aCopy = a.copy()
    pop = []
    operandPop = []
    for i in range(n):
        operandChrom = []
        for j in range(len(a)):
            if j != 0:
                bb = np.random.randint(len(b))
                operandChrom.append(b[bb])

        operandPop.append(operandChrom)
        chrom = []
        lena = len(aCopy)
        for j in range(lena):
            aa = np.random.randint(len(aCopy))
            chrom.append(aCopy[aa])
            del aCopy[aa]
        pop.append(chrom)
        aCopy = a.copy()
    return pop, operandPop

In [121]:
def operandMutation(operand, muChance, b):
    for i in range(len(operand)):
        if np.random.random()>=float(1-muChance):
            operand[i] = b[np.random.randint(len(b))]
    return operand

In [122]:
def ChromeMutation(chrome, muChance):
    for i in range(len(chrome)):
        if np.random.random() >= float(1-muChance):
            g1 = np.random.randint(len(chrome))
            g2 = np.random.randint(len(chrome))
            buf = chrome[g2]
            chrome[g2] = chrome[g1]
            chrome[g1] = buf
    return chrome

In [123]:
def crossOver(chrom1, chrom2, operand1, operand2):
    
    lenop = int(len(operand1)/2)
    operand = operand1[0:lenop]
    operand.extend(operand2[lenop:])
    chrom = []
    for i in range(len(chrom1)):
        chrom.append(chrom2[chrom1[i]-1])
    
    return chrom, operand

In [124]:
def compute(chrom, operand):
#     print(chrom[0])
    string = str(chrom[0])
    for i in range(len(operand)):
        string += str(operand[i])
        string += str(chrom[i+1])
        
    return eval(string)

In [125]:
def objectiveFunction(chrom, operand,target):
    return np.abs(compute(chrom, operand) - target)

In [135]:

def GA(a, b, target, popSize, muChance, MaxGeneration):
    nearestResponse = np.Infinity
    bestOperand = []
    bestNumbers = []
    #first population
    pop, popOperand = generatePoulation(a, b, popSize)
    #Sorting
    valueList = []
    for k in range(len(pop)):
        valueList.append(objectiveFunction(pop[k], popOperand[k],target))
    
    df = pd.DataFrame()
    df['numbers'] = pop
    df['operands'] = popOperand
    df['value'] = valueList
    df.sort_values(by='value', ascending=True, na_position='first',inplace=True)
    
    #Best first response
    if nearestResponse >= df.values[0,2]:
        nearestResponse = df.values[0,2]
        bestOperand = df.values[0,1]
        bestNumbers = df.values[0,0]
    print('best Result for generation of 0', nearestResponse, bestOperand , bestNumbers,'\n \n \n')
    
    for i in range(MaxGeneration):
        if nearestResponse ==0:
            break

        #mutation
        for j in range(len(pop)):
            if np.random.random()>=float(1-muChance):
                operandMutation(popOperand[j], muChance, b)
            if np.random.random()>=float(1-muChance):
                ChromeMutation(pop[j], muChance)        
        
        #cross over
        numberOfBestParent = int(parentEligibility*len(pop))
        eligibileParents = df.values[0:numberOfBestParent,:]

        for z in range(int(numberOfBestParent/2)):
            parent1 = np.random.randint(numberOfBestParent)
            parent2 = np.random.randint(numberOfBestParent)
            
            chrom1 = eligibileParents[parent1,0]
            chrom2 = eligibileParents[parent2,0]
            
            operand1 = eligibileParents[parent1,1]
            operand2 = eligibileParents[parent2,1]
            
            chrom, operand = crossOver(chrom1, chrom2, operand1, operand2)
            pop.append(chrom)
            popOperand.append(operand)

        
        #mutation
        for j in range(len(pop)):
            if np.random.random()>=float(1-muChance):
                operandMutation(popOperand[j], muChance, b)
            if np.random.random()>=float(1-muChance):
                ChromeMutation(pop[j], muChance)
        #Soring
        valueList = []
        for k in range(len(pop)):
            valueList.append(objectiveFunction(pop[k], popOperand[k],target))
            
        df = pd.DataFrame()
        df['numbers'] = pop
        df['operands'] = popOperand
        df['value'] = valueList
        df.sort_values(by='value', ascending=True, na_position='first',inplace=True)
        
        #Best response
        if nearestResponse >= df.values[0,2]:
            nearestResponse = df.values[0,2]
            bestOperand = df.values[0,1]
            bestNumbers = df.values[0,0]
        print('best Result for generation of ', i+1, nearestResponse, bestOperand , bestNumbers,'\n \n \n')
        pop = pop[0:popSize]
        popOperand = popOperand[0:popSize]

In [139]:
GA(a, b, target, popSize, muChance, MaxGeneration)

best Result for generation of 0 982.7876549149769 ['*', '/', '+', '-', '*', '+', '-', '-', '+', '+', '*', '+', '/', '-', '+', '*', '+', '*', '/', '-', '/', '/', '-', '+', '-', '/', '+', '+', '+', '-', '+', '-', '-', '-', '*', '+', '*', '-', '+', '+', '+', '+', '+', '-', '-', '/', '/', '-', '/'] [23, 48, 12, 10, 7, 14, 8, 43, 30, 47, 1, 6, 49, 19, 4, 17, 35, 20, 46, 37, 22, 38, 40, 9, 21, 13, 15, 16, 25, 28, 2, 42, 50, 39, 41, 44, 36, 18, 5, 29, 33, 24, 11, 32, 27, 26, 45, 34, 31, 3] 
 
 

best Result for generation of  1 982.7876549149769 ['*', '/', '+', '-', '*', '+', '-', '-', '+', '+', '*', '+', '/', '-', '+', '*', '+', '*', '/', '-', '/', '/', '-', '+', '-', '/', '+', '+', '+', '-', '+', '-', '-', '-', '*', '+', '*', '-', '+', '+', '+', '+', '+', '-', '-', '/', '/', '-', '/'] [23, 48, 12, 10, 7, 14, 8, 43, 30, 47, 1, 6, 49, 19, 4, 17, 35, 20, 46, 37, 22, 38, 40, 9, 21, 13, 15, 16, 25, 28, 2, 42, 50, 39, 41, 44, 36, 18, 5, 29, 33, 24, 11, 32, 27, 26, 45, 34, 31, 3] 
 
 

best Result

best Result for generation of  33 141.56968054279832 ['*', '+', '+', '-', '/', '/', '+', '/', '-', '+', '*', '/', '*', '-', '/', '-', '/', '/', '/', '-', '-', '/', '-', '/', '*', '+', '-', '*', '-', '/', '+', '/', '/', '-', '+', '/', '+', '/', '/', '+', '-', '-', '/', '*', '-', '*', '/', '/', '-'] [42, 29, 7, 27, 36, 5, 6, 14, 12, 32, 23, 19, 37, 50, 10, 45, 46, 20, 44, 28, 13, 40, 22, 38, 15, 33, 48, 30, 41, 9, 25, 18, 31, 8, 47, 35, 49, 17, 4, 16, 1, 21, 3, 11, 26, 24, 2, 34, 39, 43] 
 
 

best Result for generation of  34 141.56968054279832 ['*', '+', '+', '-', '/', '/', '+', '/', '-', '+', '*', '/', '*', '-', '/', '-', '/', '/', '/', '-', '-', '/', '-', '/', '*', '+', '-', '*', '-', '/', '+', '/', '/', '-', '+', '/', '+', '/', '/', '+', '-', '-', '/', '*', '-', '*', '/', '/', '-'] [42, 29, 7, 27, 36, 5, 6, 14, 12, 32, 23, 19, 37, 50, 10, 45, 46, 20, 44, 28, 13, 40, 22, 38, 15, 33, 48, 30, 41, 9, 25, 18, 31, 8, 47, 35, 49, 17, 4, 16, 1, 21, 3, 11, 26, 24, 2, 34, 39, 43] 
 
 

best R

best Result for generation of  64 134.6992568488452 ['*', '+', '+', '-', '/', '/', '+', '/', '-', '+', '*', '/', '*', '-', '/', '-', '/', '/', '/', '-', '-', '/', '-', '/', '*', '+', '-', '*', '-', '/', '+', '/', '/', '-', '+', '/', '+', '/', '/', '+', '-', '-', '/', '*', '*', '*', '/', '/', '-'] [27, 29, 7, 42, 36, 5, 6, 14, 12, 32, 23, 19, 37, 50, 10, 45, 46, 20, 44, 28, 13, 40, 22, 38, 15, 33, 48, 30, 41, 9, 25, 18, 31, 8, 47, 35, 49, 17, 4, 16, 1, 21, 3, 11, 26, 24, 2, 34, 39, 43] 
 
 

best Result for generation of  65 134.6992568488452 ['*', '+', '+', '-', '/', '/', '+', '/', '-', '-', '*', '/', '*', '-', '/', '-', '/', '/', '/', '-', '-', '/', '-', '/', '*', '-', '-', '*', '-', '/', '+', '/', '/', '-', '+', '/', '+', '/', '/', '+', '-', '-', '/', '*', '*', '*', '/', '/', '-'] [27, 29, 7, 42, 36, 5, 6, 14, 12, 32, 23, 19, 37, 50, 10, 45, 46, 20, 44, 28, 13, 40, 22, 38, 15, 33, 48, 30, 41, 9, 25, 18, 31, 8, 47, 35, 49, 17, 4, 16, 1, 21, 3, 11, 26, 24, 2, 34, 39, 43] 
 
 

best Res

best Result for generation of  99 60.777774577154105 ['/', '-', '+', '*', '-', '-', '-', '+', '*', '+', '/', '+', '/', '+', '+', '/', '/', '*', '/', '/', '+', '+', '+', '/', '+', '*', '+', '-', '/', '+', '/', '/', '-', '*', '-', '+', '-', '-', '*', '/', '/', '*', '+', '-', '*', '-', '/', '*', '-'] [36, 42, 18, 10, 11, 22, 33, 45, 12, 27, 40, 43, 48, 38, 37, 29, 9, 8, 14, 21, 35, 49, 39, 15, 46, 17, 30, 13, 7, 20, 47, 50, 25, 26, 4, 3, 23, 5, 24, 2, 1, 32, 44, 28, 31, 6, 41, 34, 16, 19] 
 
 

best Result for generation of  100 60.777774577154105 ['/', '-', '+', '*', '-', '-', '-', '+', '*', '+', '/', '+', '/', '+', '+', '/', '/', '*', '/', '/', '+', '+', '+', '/', '+', '*', '+', '-', '/', '+', '/', '/', '-', '*', '-', '+', '-', '-', '*', '/', '/', '*', '+', '-', '*', '-', '/', '*', '-'] [36, 42, 18, 10, 11, 22, 33, 45, 12, 27, 40, 43, 48, 38, 37, 29, 9, 8, 14, 21, 35, 49, 39, 15, 46, 17, 30, 13, 7, 20, 47, 50, 25, 26, 4, 3, 23, 5, 24, 2, 1, 32, 44, 28, 31, 6, 41, 34, 16, 19] 
 
 

best 

best Result for generation of  138 60.777774577154105 ['/', '-', '+', '*', '-', '-', '-', '+', '*', '+', '/', '+', '/', '+', '+', '/', '/', '*', '/', '/', '+', '+', '+', '/', '+', '*', '+', '-', '/', '+', '/', '/', '-', '*', '-', '+', '-', '-', '*', '/', '/', '*', '+', '-', '*', '-', '/', '*', '-'] [36, 42, 18, 10, 11, 22, 33, 45, 12, 27, 40, 43, 48, 38, 37, 29, 9, 8, 14, 21, 35, 49, 39, 15, 46, 17, 30, 13, 7, 20, 47, 50, 25, 26, 4, 3, 23, 5, 24, 2, 1, 32, 44, 28, 31, 6, 41, 34, 16, 19] 
 
 

best Result for generation of  139 60.777774577154105 ['/', '-', '+', '*', '-', '-', '-', '+', '*', '+', '/', '+', '/', '+', '+', '/', '/', '*', '/', '/', '+', '+', '+', '/', '+', '*', '+', '-', '/', '+', '/', '/', '-', '*', '-', '+', '-', '-', '*', '/', '/', '*', '+', '-', '*', '-', '/', '*', '-'] [36, 42, 18, 10, 11, 22, 33, 45, 12, 27, 40, 43, 48, 38, 37, 29, 9, 8, 14, 21, 35, 49, 39, 15, 46, 17, 30, 13, 7, 20, 47, 50, 25, 26, 4, 3, 23, 5, 24, 2, 1, 32, 44, 28, 31, 6, 41, 34, 16, 19] 
 
 

best

best Result for generation of  171 22.372445112411015 ['/', '-', '+', '*', '+', '-', '-', '+', '*', '+', '-', '+', '-', '*', '+', '/', '+', '*', '/', '+', '+', '+', '+', '+', '+', '-', '+', '-', '/', '+', '/', '/', '-', '-', '-', '+', '-', '-', '*', '/', '+', '*', '+', '-', '*', '-', '/', '-', '+'] [33, 17, 46, 2, 43, 8, 1, 22, 28, 21, 44, 5, 42, 25, 26, 47, 14, 38, 49, 10, 27, 45, 15, 23, 3, 30, 37, 20, 4, 41, 16, 36, 34, 19, 50, 24, 39, 12, 7, 6, 35, 48, 11, 40, 13, 18, 29, 32, 31, 9] 
 
 

best Result for generation of  172 22.372445112411015 ['/', '-', '+', '*', '+', '-', '-', '+', '*', '+', '-', '+', '-', '*', '+', '/', '+', '*', '/', '+', '+', '+', '+', '+', '+', '-', '+', '-', '/', '+', '/', '/', '-', '-', '-', '+', '-', '-', '*', '/', '+', '*', '+', '-', '*', '-', '/', '-', '+'] [33, 17, 46, 2, 43, 8, 1, 22, 28, 21, 44, 5, 42, 25, 26, 47, 14, 38, 49, 10, 27, 45, 15, 23, 3, 30, 37, 20, 4, 41, 16, 36, 34, 19, 50, 24, 39, 12, 7, 6, 35, 48, 11, 40, 13, 18, 29, 32, 31, 9] 
 
 

best

best Result for generation of  205 22.372445112411015 ['/', '-', '+', '*', '+', '-', '-', '+', '*', '+', '-', '+', '-', '*', '+', '/', '+', '-', '/', '+', '+', '+', '/', '/', '+', '-', '-', '-', '/', '+', '/', '/', '-', '-', '-', '+', '*', '-', '*', '/', '*', '*', '+', '-', '*', '-', '/', '-', '+'] [33, 17, 9, 2, 43, 8, 1, 22, 28, 21, 44, 5, 42, 39, 26, 46, 14, 38, 49, 10, 27, 45, 15, 23, 3, 30, 37, 20, 4, 41, 16, 36, 34, 19, 50, 24, 25, 12, 7, 6, 35, 48, 11, 40, 13, 18, 29, 32, 31, 47] 
 
 

best Result for generation of  206 22.372445112411015 ['/', '-', '+', '*', '+', '-', '-', '+', '*', '+', '-', '+', '-', '*', '+', '/', '+', '-', '/', '+', '+', '+', '/', '/', '+', '-', '-', '-', '/', '+', '/', '/', '-', '-', '-', '+', '*', '-', '*', '/', '*', '*', '+', '-', '*', '-', '/', '-', '+'] [33, 17, 9, 2, 43, 8, 1, 3, 28, 21, 44, 36, 42, 39, 26, 46, 14, 38, 49, 10, 27, 45, 15, 23, 22, 30, 37, 20, 4, 41, 16, 5, 34, 19, 50, 24, 25, 12, 7, 6, 35, 48, 11, 40, 13, 18, 29, 32, 31, 47] 
 
 

best

best Result for generation of  232 22.372445112411015 ['/', '-', '+', '*', '+', '-', '-', '+', '*', '+', '-', '+', '-', '*', '+', '/', '+', '-', '/', '+', '+', '+', '/', '/', '+', '-', '-', '-', '/', '+', '/', '/', '-', '-', '-', '+', '*', '-', '*', '/', '*', '*', '+', '-', '*', '-', '/', '-', '+'] [33, 17, 9, 2, 43, 8, 1, 3, 28, 21, 44, 36, 42, 39, 11, 46, 14, 38, 49, 10, 27, 45, 15, 23, 22, 30, 37, 20, 4, 41, 16, 5, 34, 19, 48, 24, 25, 12, 7, 6, 35, 50, 26, 40, 13, 18, 29, 32, 31, 47] 
 
 

best Result for generation of  233 22.372445112411015 ['/', '-', '+', '*', '+', '-', '-', '+', '*', '+', '-', '+', '-', '*', '+', '/', '+', '-', '/', '+', '+', '+', '/', '/', '+', '-', '-', '-', '/', '+', '/', '/', '-', '-', '-', '+', '*', '-', '*', '/', '*', '*', '+', '-', '*', '-', '/', '-', '+'] [33, 17, 9, 2, 43, 8, 1, 3, 28, 21, 44, 36, 42, 39, 11, 46, 14, 38, 49, 10, 27, 45, 15, 23, 22, 30, 37, 20, 4, 41, 16, 5, 34, 19, 48, 24, 25, 12, 7, 6, 35, 50, 26, 40, 13, 18, 29, 32, 31, 47] 
 
 

best